In [2]:
import torch
import torch.nn as nn
import torchvision
import pandas as pd
import numpy as np
import zipfile
import io
import pandas as pd
import math
from IPython.display import display
from collections import defaultdict

# data_preparation.ipynb created train.zip which has train.csv inside
zipped_data_path = "../data/clean_data/class-competition-cleaned.zip"
train_csv = "train_call_type_B.csv"
test_csv = "test_public.csv"

with zipfile.ZipFile(zipped_data_path, "r") as zipf:
    train_data = pd.read_csv(zipf.open(train_csv))
    test_data = pd.read_csv(zipf.open(test_csv))

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
# Split the START_LOCATION into two new columns for latitude and longitude for train set
for i, j in train_data['START_LOCATION'].items():
    split = j.split(',')
    train_data.at[i, 'START_LAT'] = split[0]
    train_data.at[i, 'START_LONG'] = split[1]
train_data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,POLYLINE,TRAVEL_TIME,START_LOCATION,MON_sin,MON_cos,DAY_sin,DAY_cos,HR_sin,HR_cos,WK_sin,WK_cos,YR_2013,START_LAT,START_LONG
0,1372637303620000596,B,NaN,7.0,20000596,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",270,"41.1599801853,-8.64198392478",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1599801853,-8.64198392478
1,1372638481620000403,B,NaN,28.0,20000403,"[[-8.584263,41.163156],[-8.584695,41.163003],[...",960,"41.1632224305,-8.58404677278",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1632224305,-8.58404677278
2,1372639960620000309,B,NaN,38.0,20000309,"[[-8.60418,41.160969],[-8.603874,41.1615],[-8....",285,"41.1607148883,-8.60424608207",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1607148883,-8.60424608207
3,1372639635620000178,B,NaN,52.0,20000178,"[[-8.613243,41.154444],[-8.612811,41.153733],[...",255,"41.1549650972,-8.61321698848",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1549650972,-8.61321698848
4,1372639973620000686,B,NaN,23.0,20000686,"[[-8.61255,41.146128],[-8.612199,41.146074],[-...",285,"41.1460158298,-8.61257471887",1.224647e-16,-1.0,-0.201299,0.97953,-0.965926,-0.258819,-0.781831,0.62349,True,41.1460158298,-8.61257471887


In [5]:
# Use only data points with CALL_TYPE B
test_data = test_data[test_data['CALL_TYPE'] == 'B']
# Split the START_LOCATION into two new columns for latitude and longitude for test set
for i, j in test_data['START_LOCATION'].items():
    split = j.split(',')
    test_data.at[i, 'START_LAT'] = split[0]
    test_data.at[i, 'START_LONG'] = split[1]
test_data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,START_LOCATION,MON_sin,MON_cos,DAY_sin,DAY_cos,HR_sin,HR_cos,WK_sin,WK_cos,YR_2013,START_LAT,START_LONG
0,T1,B,NaN,15.0,20000542,"41.1486275073,-8.58587660305",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1486275073,-8.58587660305
1,T2,B,NaN,57.0,20000108,"41.1457188058,-8.61070701502",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1457188058,-8.61070701502
2,T3,B,NaN,15.0,20000370,"41.1486275073,-8.58587660305",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1486275073,-8.58587660305
3,T4,B,NaN,53.0,20000492,"41.1412081961,-8.61401226127",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1412081961,-8.61401226127
4,T5,B,NaN,18.0,20000621,"41.1483209068,-8.61960347647",-0.866025,-0.5,0.299363,-0.954139,0.5,-0.866025,0.433884,-0.900969,False,41.1483209068,-8.61960347647


In [6]:
from sklearn.preprocessing import StandardScaler
mean_encoding = train_data.groupby('TAXI_ID')['TRAVEL_TIME'].mean().reset_index()

# Create a dictionary mapping 'TAXI_ID' to mean 'TRAVEL_TIME' value
mean_encoding_dict = dict(zip(mean_encoding['TAXI_ID'], mean_encoding['TRAVEL_TIME']))

# Replace the 'TAXI_ID' values with mean target encoding values
train_data.loc[:, 'TAXI_ID_MEAN_ENC'] = train_data['TAXI_ID'].map(mean_encoding_dict)
test_data.loc[:, 'TAXI_ID_MEAN_ENC'] = test_data['TAXI_ID'].map(mean_encoding_dict)

mean_enc_train = pd.DataFrame(train_data['TAXI_ID_MEAN_ENC'])
mean_enc_test = pd.DataFrame(test_data['TAXI_ID_MEAN_ENC'])

# Initialize StandardScaler and fit it on the mean encoding column
scaler = StandardScaler()
scaler.fit(mean_enc_train)

# Transform the mean encoding column using the fitted scaler
normalized_enc_train = scaler.transform(mean_enc_train)
normalized_enc_test = scaler.transform(mean_enc_test)

# Replace the original mean encoding column with the normalized values
train_data.loc[:, 'TAXI_ID_MEAN_ENC_NORMALIZED'] = normalized_enc_train
test_data.loc[:, 'TAXI_ID_MEAN_ENC_NORMALIZED'] = normalized_enc_test

latitude_train = pd.DataFrame(train_data['START_LAT'].astype(float))
longitude_train = pd.DataFrame(train_data['START_LONG'].astype(float))
latitude_test = pd.DataFrame(test_data['START_LAT'].astype(float))
longitude_test = pd.DataFrame(test_data['START_LONG'].astype(float))

scaler = StandardScaler()
scaler.fit(latitude_train)

normalized_latitude_train = scaler.transform(latitude_train)
normalized_latitude_test = scaler.transform(latitude_test)

scaler = StandardScaler()
scaler.fit(longitude_train)

normalized_longitude_train = scaler.transform(longitude_train)
normalized_longitude_test = scaler.transform(longitude_test)

train_data.loc[:, 'START_LAT_NORMALIZED'] = normalized_latitude_train
train_data.loc[:, 'START_LONG_NORMALIZED'] = normalized_longitude_train
test_data.loc[:, 'START_LAT_NORMALIZED'] = normalized_latitude_test
test_data.loc[:, 'START_LONG_NORMALIZED'] = normalized_longitude_test

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721030 entries, 0 to 721029
Data columns (total 23 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   TRIP_ID                      721030 non-null  int64  
 1   CALL_TYPE                    721030 non-null  object 
 2   ORIGIN_CALL                  0 non-null       float64
 3   ORIGIN_STAND                 721030 non-null  float64
 4   TAXI_ID                      721030 non-null  int64  
 5   POLYLINE                     721030 non-null  object 
 6   TRAVEL_TIME                  721030 non-null  int64  
 7   START_LOCATION               721030 non-null  object 
 8   MON_sin                      721030 non-null  float64
 9   MON_cos                      721030 non-null  float64
 10  DAY_sin                      721030 non-null  float64
 11  DAY_cos                      721030 non-null  float64
 12  HR_sin                       721030 non-null  float64
 13 

In [7]:
from sklearn.model_selection import train_test_split

# We could totally change this. Utilization of these just probably requires further preprocessing.
ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION = ['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'POLYLINE', 'START_LOCATION', 'TAXI_ID_MEAN_ENC', 'TAXI_ID', 'START_LAT', 'START_LONG']

train_data_sample = train_data.sample(frac=0.8, random_state=420) # frac is used to control percentage of train data used
X = train_data_sample.drop("TRAVEL_TIME", axis=1)
X = X.loc[:, ~X.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]
y = train_data_sample["TRAVEL_TIME"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

test_features = test_data.loc[:, ~test_data.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]

In [13]:
X_train.head()

,ORIGIN_STAND,MON_sin,MON_cos,DAY_sin,DAY_cos,HR_sin,HR_cos,WK_sin,WK_cos,YR_2013,TAXI_ID_MEAN_ENC_NORMALIZED,START_LAT_NORMALIZED,START_LONG_NORMALIZED
170697,21.0,-1.000000e+00,-1.836970e-16,-0.937752,0.347305,0.866025,0.500000,0.974928,-0.222521,True,-0.581915,0.391930,-0.700204
443500,15.0,8.660254e-01,5.000000e-01,0.790776,-0.612106,-0.500000,-0.866025,0.781831,0.623490,False,0.713990,-0.687404,1.332571
680081,49.0,1.224647e-16,-1.000000e+00,0.790776,-0.612106,0.866025,0.500000,0.974928,-0.222521,False,1.026098,2.035271,0.557289
404924,26.0,5.000000e-01,8.660254e-01,-0.790776,-0.612106,0.258819,0.965926,0.000000,1.000000,False,-0.081409,0.275207,1.616427
159763,20.0,-1.000000e+00,-1.836970e-16,-0.790776,-0.612106,0.866025,0.500000,-0.433884,-0.900969,True,-0.168471,0.923853,-0.570558


In [8]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std

rf = RandomForestRegressor(random_state=420, max_features='sqrt', n_estimators = 1200, min_samples_leaf=4, min_samples_split=10, max_depth=60, bootstrap=True, n_jobs=-1)
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate the model and collect the scores
# n_scores = cross_val_score(rf, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [10]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

root_mean_squared_error(y_test, y_pred)

336.2240000423286

In [12]:
def test_prediction_to_csv(y_pred, outfile_name):
	output_df = pd.DataFrame(test_data["TRIP_ID"])
	output_df["TRAVEL_TIME"] = y_pred
	output_df.head()
	output_df.to_csv(f'../guesses/{outfile_name}', index=False)
	
y_pred = rf.predict(test_features)
test_prediction_to_csv(y_pred, "predicting_random_forest_calltype_B.csv")

### XGBoost

In [ ]:
from xgboost import XGBRFRegressor
model = XGBRFRegressor(n_estimators=1200, colsample_bynode=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
root_mean_squared_error(y_test, y_pred)

341.47698978709155

In [15]:
from xgboost import XGBRFRegressor
from numpy import arange
# WARNING: TAKES A WHILE (9 MIN ON M1 PRO CPU)
n_trees = [10, 50, 100, 500, 1200]
models = dict()
for v in n_trees:
    print('Num trees: ', v)
    for b in arange(0.1, 1.1, 0.2):
        models[str(v)] = XGBRFRegressor(n_estimators=v, colsample_bynode=b)
        models[str(v)].fit(X_train, y_train)
        y_pred = models[str(v)].predict(X_test)
        print('colsample_bynode: ', b, '| RMSE: ', root_mean_squared_error(y_test, y_pred))

Num trees:  10
colsample_bynode:  0.1 | RMSE:  346.9507266613267
colsample_bynode:  0.30000000000000004 | RMSE:  342.83808895812
colsample_bynode:  0.5000000000000001 | RMSE:  341.6577324724118
colsample_bynode:  0.7000000000000001 | RMSE:  341.5502965564866
colsample_bynode:  0.9000000000000001 | RMSE:  341.5924782229254
Num trees:  50
colsample_bynode:  0.1 | RMSE:  346.4934949742217
colsample_bynode:  0.30000000000000004 | RMSE:  342.1967420702023
colsample_bynode:  0.5000000000000001 | RMSE:  341.42033714452714
colsample_bynode:  0.7000000000000001 | RMSE:  341.4784772429874
colsample_bynode:  0.9000000000000001 | RMSE:  341.7003155034272
Num trees:  100
colsample_bynode:  0.1 | RMSE:  346.47162961842014
colsample_bynode:  0.30000000000000004 | RMSE:  342.1709450129984
colsample_bynode:  0.5000000000000001 | RMSE:  341.32954181537235
colsample_bynode:  0.7000000000000001 | RMSE:  341.44826434673575
colsample_bynode:  0.9000000000000001 | RMSE:  341.71542502326554
Num trees:  500
co

In [14]:
y_pred = model.predict(test_features)
test_prediction_to_csv(y_pred, "predicting_rf_xgboost_calltype_B.csv")

NameError: name 'model' is not defined